<a href="https://colab.research.google.com/github/GrigoreGeorgeAlexandru/Colab-projects/blob/main/Lab_2_APD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Algoritmi Paraleli si Distribuiti - Lab 2

## 1. Barrier.

Scrieti un program paralel care sa creeze mai multe procese care se sincronizeaza folosind bariere in felul urmator:
* procesul zero (`root`) simuleaza o activitate dormind 4 secunde dupa care scrie un mesaj pe ecran;
* toate celelalte procesoare asteapta sa termine procesorul zero folosind o bariera, apoi scriu un mesaj pe ecran avertizand ca se pregatesc de lucru (de fapt se pregatesc de `sleep`)
* apoi, toate procesoarele simuleaza o activitate aleatoare intre 0 si 10 secunde folosind `sleep`, iar dupa terminarea activitatii scriu un mesaj pe ecran

Apoi, toate procesoarele apeleaza rutina `Barrier` pentru a se sincroniza unele cu altele.

La final, dupa deblocarea din bariera, procesul `root` conclude ca toate procesele s-au terminat si scrie un mesaj pe ecran in acest sens.

In [ ]:
# Importurile
import multiprocessing
import time
import random
import os

In [ ]:
def process_task(pid, barrier1, barrier2):
    """
    Args:
        pid (int): Numarul procesului - 0 e `root`.
        barrier1 (multiprocessing.Barrier): Prima bariera.
        barrier2 (multiprocessing.Barrier): A 2-a bariera.
    """
    # Simululez procesului 0
    if pid == 0:
        print("Incepe procesul 0 (`root`). Incep un task care dureaza 4 secunde.")
        time.sleep(4)
        print("Procesul 0 (`root`) a terminat job-ul de 4 secunde.")

    # Astept la prima bariera.
    barrier1.wait()

    # Fac un print sa anunte ca au terminat de asteptat
    if pid != 0:
        print(f"Procesul {pid} a terminat de asteptat.")

    # Generez un timp random cat sa dureze fiecare proces (fac sleep).
    sleep_duration = random.uniform(0, 10)
    print(f"Procesul {pid} va dura {sleep_duration:.2f} secunde.")
    time.sleep(sleep_duration)
    print(f"Procesul {pid} si-a terminat treaba.")

    # Folosesc o alta bariera ca sa sincronizez toate procesele
    print(f"Procesul {pid} asteapta la a 2-a bariera.")
    barrier2.wait()

    # Daca sunt procesul 0, atunci afisez mesajul dorit
    if pid == 0:
        print("========================================================")
        print("===    Procesul 0: Toate procesele s-au terminat.    ===")
        print("========================================================")


In [ ]:
# Definesc numarul de procese
num_procese = 5  # 1 (root) + 4 = 5

print(f"Am inceput. Numarul de procese = {num_procese}.\n")

barrier1 = multiprocessing.Barrier(num_procese)
barrier2 = multiprocessing.Barrier(num_procese)

# Fac o lista de procese ca dupa sa ii fac join.
processes = []

# Creez fiecare dintre cele 5 procese.
for i in range(num_procese):
    process = multiprocessing.Process(
        target=process_task,
        args=(i, barrier1, barrier2)
    )
    processes.append(process)
    process.start()  # Apelez `process_task` pentru procesul process

for process in processes:
    process.join()

Am inceput. Numarul de procese = 5.

Incepe procesul 0 (`root`). Incep un task care dureaza 4 secunde.
Procesul 0 (`root`) a terminat job-ul de 4 secunde.
Procesul 0 va dura 1.92 secunde.
Procesul 2 a terminat de asteptat.Procesul 4 a terminat de asteptat.
Procesul 1 a terminat de asteptat.Procesul 3 a terminat de asteptat.
Procesul 2 va dura 10.00 secunde.

Procesul 1 va dura 3.01 secunde.
Procesul 3 va dura 0.66 secunde.Procesul 4 va dura 9.93 secunde.


Procesul 3 si-a terminat treaba.
Procesul 3 asteapta la a 2-a bariera.
Procesul 0 si-a terminat treaba.
Procesul 0 asteapta la a 2-a bariera.
Procesul 1 si-a terminat treaba.
Procesul 1 asteapta la a 2-a bariera.
Procesul 4 si-a terminat treaba.
Procesul 4 asteapta la a 2-a bariera.
Procesul 2 si-a terminat treaba.
Procesul 2 asteapta la a 2-a bariera.
===    Procesul 0: Toate procesele s-au terminat.    ===


## 2. Broadcast

Scrieti un program paralel in care procesul zero (`root`) foloseste rutina de broadcast pentru a trimite un dictionar cu doua chei `key1` : `[3, 24.62, 9+4j]`, `key2` : ( `fmi`, `unibuc`) catre toate procesele.

Apoi scrieti o versiune care trimite un vector in loc de dictionar.

In [ ]:
!pip install mpi4py

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 466.3/466.3 kB 8.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for mpi4py: filename=mpi4py-4.0.3-cp311-cp311-linux_x86_64.whl size=4442045 sha256=1c41bc8c425b2a6fd7f86a6b8477a7b1e50167c5a1ca109960c8c917921f9fc7
  Stored in directory: /root/.cache/pip/wheels/5c/56/17/bf6ba37aa971a191a8b9eaa188bf5ec855b8911c1c56fb1f84
Successfully built mpi4py


In [ ]:
%%writefile broadcast_task_1.py

# Folosesc libraria Message Passing Interface (MPI) din python.
from mpi4py import MPI

comm = MPI.COMM_WORLD
size = comm.Get_size() # Numarul total de procese
rank = comm.Get_rank() # ID-ul unui proces, de la 0 la size-1


# Dictionarul e definit pentru procesul 0, care il trimite si celorlalte
# procese.
if rank == 0:
    dictionar = {
        'key1': [3, 24.62, 9+4j],
        'key2': ('fmi', 'unibuc')
    }
    print(f"Procesul 0 (root) transmite dictionarul:\n{dictionar}\n")
else:
    # Pentru celelalte procese initial e null
    dictionar = None

# root = 0 spune practic ca procesul 0 e cel care face send.
received_dict = comm.bcast(dictionar, root=0)

print(f"Procesul {rank} a primit dictionarul:\n{received_dict}")


Overwriting broadcast_task_1.py


In [ ]:
!mpiexec -n 4 --allow-run-as-root --oversubscribe python broadcast_task_1.py

Procesul 0 (root) transmite dictionarul:
{'key1': [3, 24.62, (9+4j)], 'key2': ('fmi', 'unibuc')}

Procesul 0 a primit dictionarul:
{'key1': [3, 24.62, (9+4j)], 'key2': ('fmi', 'unibuc')}
Procesul 2 a primit dictionarul:
{'key1': [3, 24.62, (9+4j)], 'key2': ('fmi', 'unibuc')}
Procesul 1 a primit dictionarul:
{'key1': [3, 24.62, (9+4j)], 'key2': ('fmi', 'unibuc')}
Procesul 3 a primit dictionarul:
{'key1': [3, 24.62, (9+4j)], 'key2': ('fmi', 'unibuc')}


In [ ]:
%%writefile broadcast_task_2.py

# Folosesc libraria Message Passing Interface (MPI) din python.
from mpi4py import MPI

comm = MPI.COMM_WORLD
size = comm.Get_size() # Numarul total de procese
rank = comm.Get_rank() # ID-ul unui proces, de la 0 la size-1


# Vectorul e definit pentru procesul 0, care il trimite si celorlalte procese.
if rank == 0:
    vector = [1, 2, 3]
    print(f"Procesul 0 (root) transmite vectorul:\n{vector}\n")
else:
    # Pentru celelalte procese initial e null
    vector = None

# root = 0 spune practic ca procesul 0 e cel care face send.
received_vect = comm.bcast(vector, root=0)

print(f"Procesul {rank} a primit vector:\n{received_vect}")


Overwriting broadcast_task_2.py


In [ ]:
!mpiexec -n 4 --allow-run-as-root --oversubscribe python broadcast_task_2.py

Procesul 0 (root) transmite vectorul:
[1, 2, 3]

Procesul 0 a primit vector:
[1, 2, 3]
Procesul 1 a primit vector:
[1, 2, 3]
Procesul 2 a primit vector:
[1, 2, 3]
Procesul 3 a primit vector:
[1, 2, 3]


## 3. Scatter

Procesul zero (`root`) creaza o matrice $p \times p$ de numere reale (nr de linii egal cu nr de procesoare din comunicator). Scrieti un program paralel in care procesul zero foloseste rutina de distributie personalizata (`scatter`) pentru a trimite fiecarui procesor cu id `rank` din comunicator un mesaj compus din coloana cu indexul rank si respectiv linia cu indexul `rank`. Afisati pentru verificare.

In [ ]:
%%writefile scatter_task.py

from mpi4py import MPI
import numpy as np

comm = MPI.COMM_WORLD
size = comm.Get_size() # Numarul total de procese
rank = comm.Get_rank() # ID-ul unui proces, de la 0 la size-1

# Numarul de linii = nr. de procesoare
p = size

if rank == 0:
    # In procesul se creeaza matricea de p*p.
    matrix = np.arange(p * p, dtype=np.float64).reshape(p, p)
    print(f"Procesul 0 a creat matrice de {p}x{p}:\n{matrix}\n")

    for i in range(p):
        row_to_send = np.copy(matrix[i, :])
        col_to_send = np.copy(matrix[:, i])

        message = (row_to_send, col_to_send)

        print(f"Procesul 0 trimite linia si coloana {i} to procesului {i}.")
        comm.send(message, dest=i)

else:
    # Primesc mesajul de la procesul 0
    received_message = comm.recv(source=0)
    received_row, received_col = received_message

    # Afisez pentru verificare
    print(f"Procesul {rank} a primi:")
    print(f"   * Linia {rank}: {received_row}")
    print(f"   * Coloana {rank}: {received_col}")

Writing scatter_task.py


In [ ]:
!mpiexec -n 4 --allow-run-as-root --oversubscribe python scatter_task.py

Traceback (most recent call last):
  File "/content/scatter_task.py", line 2, in <module>
    from mpi4py import MPI
ModuleNotFoundError: No module named 'mpi4py'
--------------------------------------------------------------------------
Primary job  terminated normally, but 1 process returned
a non-zero exit code. Per user-direction, the job has been aborted.
--------------------------------------------------------------------------
Traceback (most recent call last):
  File "/content/scatter_task.py", line 2, in <module>
    from mpi4py import MPI
ModuleNotFoundError: No module named 'mpi4py'
Traceback (most recent call last):
  File "/content/scatter_task.py", line 2, in <module>
    from mpi4py import MPI
ModuleNotFoundError: No module named 'mpi4py'
Traceback (most recent call last):
  File "/content/scatter_task.py", line 2, in <module>
    from mpi4py import MPI
ModuleNotFoundError: No module named 'mpi4py'
-------------------------------------------------------------------------

## 4. Inmultire matrice-vector pe inel.

Presupunem ca nodurile din comunicator (in numar de $p$) sunt distribuite pe o topologie de tip inel. Procesul zero (`root`) stocheaza o matricile $A \in \mathbb{R}^{n \times n}$ si $x \in \mathbb{R}^n$.

*  Realizati o distributie echilibrata a tablourilor astfel incat nodul $i$ sa stocheze (bloc-)coloana $i$ din $A$ si (bloc-)componenta $i$ din $x$.

* Scrieti o procedura care realizeaza inmultirea eficienta matrice-vector in paralel. In urma unei difuzari generale pe inel, rezultatul va fi acumulat in fiecare nod din comunicator.

In [ ]:
%%writefile matvec_ring.py

from mpi4py import MPI
import numpy as np

comm = MPI.COMM_WORLD
size = comm.Get_size() # Numarul total de procese
rank = comm.Get_rank() # ID-ul unui proces, de la 0 la size-1

# Am luat, de exemplu n = 4 (si atunci numarul de procese va trebuie sa fie 1,
# 2 sau 4).
n = 4

if rank == 0:
    print(f"Numar procese (p): {size}, Dimensiune matrice (n): {n}\n")

    # Generez aleator matricea si vectorul
    A = np.random.randint(5, size = (n, n))
    x = np.random.randint(5, size = n)

    print(f"Matricea A:\n{A}")
    print(f"Vectorul x:\n{x}\n")

    # Calculul "normal" pentru verificare
    expected_y = np.dot(A, x)

    d = n // size

    # Root pregateste blocurile pentru a le imparti celorlalte procese
    A_cols = [A[:, i:i + d].copy() for i in range(0, n, d)]
    x_parts = [x[i:i + d] for i in range(0, n, d)]

    # Pastreaza primul bloc pentru el
    local_A_block = A_cols[0]
    local_x_block = x_parts[0]

    # Trimite restul blocurilor vecinului din dreapta
    if size > 1:
        comm.send({'A': A_cols[1:], 'x': x_parts[1:]}, dest=1)

else:
    # Fiecare proces primeste de la vecinul din stanga
    received_data = comm.recv(source=rank - 1)

    # Pastreaza primul bloc din ce a primit
    local_A_block = received_data['A'][0]
    local_x_block = received_data['x'][0]

    # Daca nu ești ultimul proces, trimite restul mai departe la dreapta
    if rank < size - 1:
        remaining_data = {'A': received_data['A'][1:], 'x': received_data['x'][1:]}
        comm.send(remaining_data, dest=rank + 1)


# Fiecare proces 'i' calculează "componenta: locala
partial_y = np.dot(local_A_block, local_x_block)

# Adunarea rezultatelor ("Reduce" manual pe inel)
if rank > 0:
    accumulated_sum = partial_y.copy()

    # Daca nu ești ultimul proces (p-1), primește suma de la dreapta
    if rank < size - 1:
        sum_from_right = comm.recv(source=rank + 1)
        accumulated_sum += sum_from_right

    # Trimite suma acumulata la stanga
    comm.send(accumulated_sum, dest=rank - 1)

else:
    final_result = partial_y.copy()

    if size > 1:
        sum_from_others = comm.recv(source=1)
        final_result += sum_from_others

# Verificare
if rank == 0:
    print(f"Rezultat final (paralel): {final_result}")
    print(f"Rezultatul așteptat: {expected_y}\n")

Overwriting matvec_ring.py


In [ ]:
!mpiexec -n 4 --allow-run-as-root --oversubscribe python matvec_ring.py